# Stock Price Direction Prediction with 2D CNN
- In this problem, we will integrate technical analysis indicators into an original time series, which will help us convert the original series into 2D images.
- We will use the following technical indicators from the TA-Lib library in Python: MACD, RSI, CMO, MOM, Bollinger Bands, and SMA. 
- Technical analysis indicators are financial indicators that guide traders about the market.
- We will use AAPL close prices from the "yfinance" library.
- We will use the historical 10-day closing price, build up a mxn image by calculating technical indicators, and predict the direction for the next day, whether the price will go up or down.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import callbacks
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [2]:
# Download AAPL data with Volume included
data = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
data = data[['Close', 'Volume']].reset_index()

# Split into train (first half of 2024) and test (second half of 2024)
train_df = data[(data['Date'] >= "2024-01-01") & (data['Date'] <= "2024-07-31")]
test_df = data[(data['Date'] > "2024-07-31") & (data['Date'] <= "2025-01-01")]

# Drop NaNs (if applicable)
train_df = train_df.dropna().reset_index(drop=True)
test_df = test_df.dropna().reset_index(drop=True)

# Print the shapes of the train and test datasets
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

[*********************100%***********************]  1 of 1 completed

Train shape: (146, 3)
Test shape: (106, 3)


In [3]:
train_df.head()

Price,Date,Close,Volume
Ticker,,AAPL,AAPL
0,2024-01-02 00:00:00+00:00,185.639999,82488700
1,2024-01-03 00:00:00+00:00,184.250000,58414500
2,2024-01-04 00:00:00+00:00,181.910004,71983600
3,2024-01-05 00:00:00+00:00,181.179993,62303300
4,2024-01-08 00:00:00+00:00,185.559998,59144500


In [4]:
# Generate features, such as MACD, RSI, CMO, MOM, Bollinger Bands, and SMA, by using TA-Lib
def calculate_indicators(data):
    # Ensure the 'Close' column is a NumPy array
    close_prices = data['Close'].values.flatten()  # Convert to 1D array

    # Calculate indicators
    data['RSI'] = talib.RSI(close_prices, timeperiod=14)
    data['CMO'] = talib.CMO(close_prices, timeperiod=14)
    data['MOM'] = talib.MOM(close_prices, timeperiod=10)
    macd, macd_signal, _ = talib.MACD(close_prices, fastperiod=12, slowperiod=26, signalperiod=9)
    data['MACD'] = macd - macd_signal  # Difference between MACD and Signal line
    
    # Bollinger Bands (adds three new columns)
    data['BB_Upper'], data['BB_Middle'], data['BB_Lower'] = talib.BBANDS(
        close_prices, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
    )
    
    # Simple Moving Average (SMA)
    data['SMA'] = talib.SMA(close_prices, timeperiod=14)

    return data

# Apply to train and test datasets
train_data = calculate_indicators(train_df)
test_data = calculate_indicators(test_df)

# Drop rows with missing values
train_data = train_data.dropna().reset_index(drop=True)
test_data = test_data.dropna().reset_index(drop=True)

print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (113, 11)
Test shape: (73, 11)


In [5]:
train_data.head()

Price,Date,Close,Volume,RSI,CMO,MOM,MACD,BB_Upper,BB_Middle,BB_Lower,SMA
Ticker,,AAPL,AAPL,,,,,,,,
0,2024-02-20 00:00:00+00:00,181.559998,53665600,37.100702,-25.798597,-6.119995,-1.507757,195.837398,188.038998,180.240598,186.052856
1,2024-02-21 00:00:00+00:00,182.320007,41529700,39.522843,-20.954314,-6.979996,-1.388364,194.846091,187.395999,179.945908,185.904286
2,2024-02-22 00:00:00+00:00,184.369995,52292200,45.607165,-8.785670,-5.040009,-1.114930,193.687709,186.889499,180.091288,185.726428
3,2024-02-23 00:00:00+00:00,182.520004,45119700,41.545128,-16.909745,-5.800003,-1.005628,192.477559,186.306999,180.136440,185.488571
4,2024-02-26 00:00:00+00:00,181.160004,40867400,38.808640,-22.382720,-7.690002,-0.971367,191.628937,185.743999,179.859062,185.022858


In [6]:
# Label the data based on the direction of the next day’s price movement
train_data['IsUp'] = (train_data['Close'].shift(-1) > train_data['Close']).astype(int)
test_data['IsUp'] = (test_data['Close'].shift(-1) > test_data['Close']).astype(int)

# Remove the last rows since they have no valid future value
train_data = train_data[:-1].reset_index(drop=True)
test_data = test_data[:-1].reset_index(drop=True)

print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (112, 12)
Test shape: (72, 12)


In [7]:
train_data.head(20)

Price,Date,Close,Volume,RSI,CMO,MOM,MACD,BB_Upper,BB_Middle,BB_Lower,SMA,IsUp
Ticker,,AAPL,AAPL,,,,,,,,,
0,2024-02-20 00:00:00+00:00,181.559998,53665600,37.100702,-25.798597,-6.119995,-1.507757,195.837398,188.038998,180.240598,186.052856,1
1,2024-02-21 00:00:00+00:00,182.320007,41529700,39.522843,-20.954314,-6.979996,-1.388364,194.846091,187.395999,179.945908,185.904286,1
2,2024-02-22 00:00:00+00:00,184.369995,52292200,45.607165,-8.785670,-5.040009,-1.114930,193.687709,186.889499,180.091288,185.726428,0
3,2024-02-23 00:00:00+00:00,182.520004,45119700,41.545128,-16.909745,-5.800003,-1.005628,192.477559,186.306999,180.136440,185.488571,0
4,2024-02-26 00:00:00+00:00,181.160004,40867400,38.808640,-22.382720,-7.690002,-0.971367,191.628937,185.743999,179.859062,185.022858,1
5,2024-02-27 00:00:00+00:00,182.630005,54318900,43.166200,-13.667600,-4.519989,-0.803119,190.634779,185.289000,179.943221,184.546429,0
6,2024-02-28 00:00:00+00:00,181.419998,48953900,40.603091,-18.793819,-3.619995,-0.728911,190.400367,184.958000,179.515634,183.975715,0
7,2024-02-29 00:00:00+00:00,180.750000,136682600,39.214604,-21.570791,-3.399994,-0.681413,190.517043,184.775500,179.033958,183.435000,0
8,2024-03-01 00:00:00+00:00,179.660004,73488000,36.997958,-26.004084,-4.199997,-0.678396,190.482752,184.415501,178.348249,182.778571,0


In [8]:
# Get rid of the multi-index close price and volume columns column and create new ones
train_data["Close Price"] = train_data["Close"].values
test_data["Close Price"] = test_data["Close"].values

train_data["Volume"] = train_data["Volume"].values
test_data["Volume"] = test_data["Volume"].values

# Set a new column order
required_columns = [
    "RSI", "CMO", "MOM", "MACD", "BB_Upper", "BB_Middle", 
    "BB_Lower", "SMA", "Close Price", "Volume", "IsUp",
]
new_train = train_data[required_columns].copy()
new_test = test_data[required_columns].copy()

# Flatten Multi-Index columns
new_train.columns = [' '.join(col).strip() for col in new_train.columns.values]
new_test.columns = [' '.join(col).strip() for col in new_test.columns.values]

# Separate the features and target column
train_features = new_train.drop(columns=["IsUp"])
test_features = new_test.drop(columns=["IsUp"])

# Normalize the feature columns
scaler = StandardScaler()
scaler.fit(train_features)
train_features_normalized = scaler.transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Re-attach the 'Target' column to the normalized data
new_train_normalized = pd.DataFrame(train_features_normalized, columns=train_features.columns)
new_train_normalized['IsUp'] = new_train['IsUp'].values

new_test_normalized = pd.DataFrame(test_features_normalized, columns=test_features.columns)
new_test_normalized['IsUp'] = new_test['IsUp'].values

print(f"New train shape: {new_train_normalized.shape}")
print(f"New test shape: {new_test_normalized.shape}")

New train shape: (112, 11)
New test shape: (72, 11)


In [9]:
new_train_normalized.head()

,RSI,CMO,MOM,MACD,BB_Upper,BB_Middle,BB_Lower,SMA,Close Price,Volume AAPL,IsUp
0,-1.183392,-1.183392,-0.908452,-1.602668,-0.135256,0.016017,0.228802,-0.146019,-0.444506,-0.369492,1
1,-1.020464,-1.020464,-0.992853,-1.484253,-0.182758,-0.020890,0.208857,-0.153885,-0.408250,-0.754344,1
2,-0.611193,-0.611193,-0.802461,-1.213060,-0.238266,-0.049962,0.218697,-0.163301,-0.310458,-0.413045,0
3,-0.884432,-0.884432,-0.877047,-1.104653,-0.296255,-0.083397,0.221752,-0.175894,-0.398710,-0.640498,0
4,-1.068506,-1.068506,-1.062534,-1.070673,-0.336920,-0.115712,0.202980,-0.200551,-0.463587,-0.775347,1


In [10]:
n_features = new_train_normalized.shape[1] - 1 # exclude targets (IsUp) in sequences
window_size = 5 # sequence length in days

# Create widthxheight Images
# Combine the window_size days of closing prices and technical indicators into a mxn matrix
def create_images(data, window_size=window_size):

    images, labels = list(), list()

    for i in range(len(data) - window_size):
        img = data.iloc[i:(i+window_size), :-1].values # exclude targets (IsUp) in sequences
        label = data.iloc[(i+window_size), -1]
        
        if img.shape == (window_size, n_features):
            images.append(img)
            labels.append(label)
        else:
            print(f"Expected a {(window_size, n_features)} image, but current image shape is: {img.shape}")
    
    return np.array(images), np.array(labels)

# Use the normalized data
train_images, train_labels = create_images(new_train_normalized)
test_images, test_labels = create_images(new_test_normalized)

print(f"Train images: {train_images.shape} & Train labels: {train_labels.shape}")
print(f"Test images: {test_images.shape} & Test labels: {test_labels.shape}")

Train images: (107, 5, 10) & Train labels: (107,)
Test images: (67, 5, 10) & Test labels: (67,)


In [11]:
# Define a CNN model
cnn_model = Sequential([
    # First convolutional layer
    Conv2D(64, (2, 2), activation='relu', input_shape=(window_size, n_features, 1)),
    BatchNormalization(),
    Conv2D(128, (2, 2), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (2, 2), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(), # Flatten the output for the dense layer
    Dense(128, activation='relu'),
    Dropout(0.3), # Dropout layer to prevent overfitting
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid') # Output layer (binary classification)
])

# Compile the model with an initial learning rate
cnn_model.compile(
    optimizer=Adam(learning_rate=1e-4), 
    loss='binary_crossentropy', 
    metrics=['accuracy'],
)
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 9, 64)          320       
                                                                 
 batch_normalization (Batch  (None, 4, 9, 64)          256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 8, 128)         32896     
                                                                 
 batch_normalization_1 (Bat  (None, 3, 8, 128)         512       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 7, 256)         131328    
                                                                 
 batch_normalization_2 (Bat  (None, 2, 7, 256)         1

In [12]:
# Reshape the images to match the required input shape (10, 10, 1) for the CNN model
final_train_images = train_images.reshape(-1, window_size, n_features, 1)
final_test_images = test_images.reshape(-1, window_size, n_features, 1)

# Train the CNN model
history = cnn_model.fit(
    final_train_images, 
    train_labels, 
    epochs=30, 
    batch_size=24, 
    validation_data=(final_test_images, test_labels), 
    verbose=1,
)

Epoch 1/30
5/5 [==============================] - 5s 177ms/step - loss: 0.8317 - accuracy: 0.5234 - val_loss: 0.6933 - val_accuracy: 0.4925
Epoch 2/30
5/5 [==============================] - 0s 50ms/step - loss: 0.7094 - accuracy: 0.5888 - val_loss: 0.6904 - val_accuracy: 0.5672
Epoch 3/30
5/5 [==============================] - 0s 39ms/step - loss: 0.7065 - accuracy: 0.6355 - val_loss: 0.6887 - val_accuracy: 0.6119
Epoch 4/30
5/5 [==============================] - 0s 37ms/step - loss: 0.6563 - accuracy: 0.6542 - val_loss: 0.6878 - val_accuracy: 0.6119
Epoch 5/30
5/5 [==============================] - 0s 45ms/step - loss: 0.6080 - accuracy: 0.6262 - val_loss: 0.6884 - val_accuracy: 0.5373
Epoch 6/30
5/5 [==============================] - 0s 42ms/step - loss: 0.6666 - accuracy: 0.6542 - val_loss: 0.6892 - val_accuracy: 0.5672
Epoch 7/30
5/5 [==============================] - 0s 44ms/step - loss: 0.5598 - accuracy: 0.6822 - val_loss: 0.6897 - val_accuracy: 0.5075
Epoch 8/30
5/5 [==========

In [13]:
loss, accuracy = cnn_model.evaluate(final_test_images, test_labels, verbose=1)
print(f"Test Accuracy: {(accuracy*100):.2f}%")

3/3 [==============================] - 0s 13ms/step - loss: 0.6913 - accuracy: 0.5224
Test Accuracy: 52.24%


**Discussion:** The test accuracy of 52.24% is relatively low, only slightly better than random guessing, which would yield around 50% accuracy in a binary classification problem. Several factors could explain this performance:
- The technical indicators used may not be sufficient or directly relevant for predicting the target, which is whether the Bitcoin price will go up or down.
- The training set may be too small to capture meaningful patterns between features and the target variable.
- The training period should include bear, bull, and neutral market conditions to make meaningful predictions.
- The basic CNN architecture without hyperparameter tuning may not provide enough capacity or optimization to learn complex relationships in the data effectively.

# END